In [41]:
import os
import torch

import joblib

num_cores = os.cpu_count()
num_GPUs = torch.cuda.device_count()

print('# Cores:' + str(num_cores))
print('# GPUs: ' + str(num_GPUs))

# Get the available GPUs directly as a list
print(f"Available GPUs: {list(range(torch.cuda.device_count()))}")

print('Visible GPUs Indices: ' + str(os.environ.get('CUDA_VISIBLE_DEVICES', 'All GPUs are visible')))

# Cores:8
# GPUs: 0
Available GPUs: []
Visible GPUs Indices: 0


In [42]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [43]:
%load_ext autoreload
%autoreload 2

from utils.create_dataset_class import DataSet
from utils.multiclass_NN import multiclass_NN
from utils.split_dataset import split
from utils.scale_graph_features import scale

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
SEED = 103
db_file = 'data_preprocessing/db.csv'

SPLIT_RATIO = '0.4,0.3,0.3' # Train, Val, Test
# SPLIT_RATIO = '0.7,0.3' # Train, Val, Test

SMILES = 'data_preprocessing/SMILES.txt'
DESCRIPTORS = 'monomer_data/unique_descriptors.json'

LABELNAME = 'binary_class' # Multi-class will not work for mixed=False (test set are polymers) because polymers don't represent all classes 
TASK = 'classification'
MODEL = 'MPNN'

NUM_EPOCHS = 4
NUM_WORKERS = num_cores

MODEL_PATH = 'past_trials/' + MODEL + '/' + str(NUM_EPOCHS) + '_epochs'

SAVE_MODEL = True
SAVE_OPT = True
SAVE_CONFIG = True

CUSTOM_PARAMS = {}

In [45]:
split_db = split(db_file, SEED, SPLIT_RATIO, MIXED = True)

In [46]:
features = scale(split_db['train'], SMILES, DESCRIPTORS)

In [47]:
dataset = DataSet(db_file, features, split_db, LABELNAME, TASK, MODEL)

In [ ]:
test = multiclass_NN(dataset, MODEL, NUM_EPOCHS, NUM_WORKERS, DESCRIPTORS, CUSTOM_PARAMS, MODEL_PATH, SAVE_MODEL, SAVE_OPT, SAVE_CONFIG)
test.main()

In [15]:
joblib.dump(features, MODEL_PATH + '/features.pkl')

['features.pkl']

In [48]:
joblib.dump(split_db, 'split_db.pkl')
joblib.dump(features, 'features.pkl')
joblib.dump(dataset, 'dataset.pkl')

['dataset.pkl']